In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from concurrent.futures import ThreadPoolExecutor
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

caps = DesiredCapabilities.CHROME
caps["pageLoadStrategy"] = "none"

In [10]:
class DaejinCrawler:
    def __init__(self, username, password):
        self.driver = webdriver.Chrome(options=self.ChromeSetup())
        self.username = username
        self.password = password
        self.Course = {}
        self.Require = {}
        self.consultng_total = {}
        self.consulting_detail = {}
        self.sub_elements = None

    def __del__(self):
        self.driver.quit()

    def ChromeSetup(self):
        options = webdriver.ChromeOptions()
        options.add_argument("--headless")
        options.add_argument('--ignore-ssl-errors=yes')
        options.add_argument('--ignore-certificate-errors')
        options.add_argument('--disable-dev-shm-usage')
        options.add_argument('--disable-images')
        options.add_argument("--no-sandbox")
        options.add_argument("--disable-extensions")
        options.add_argument("--disable-popup-blocking")

        return options
    
    def closepopup(driver):
        main = driver.window_handles
        for i in main:
            if i != main[0]:
                driver.switch_to.window(i)
                driver.close()

        driver.switch_to.window(main[0])

    def crawl_consulting(self):
        driver = self.driver
        
        twin_url = 'https://together.daejin.ac.kr/clientMain/a/t/main.do'
        driver.get(twin_url)

        self.closepopup(driver)

        driver.find_element(By.CLASS_NAME, 'btn_log.btn_login').click()
        time.sleep(1)
        
        username_input = driver.find_element(By.ID, 'userId')
        password_input = driver.find_element(By.ID, 'userPw')

        username_input.send_keys(self.username)
        password_input.send_keys(self.password)
        driver.find_element(By.ID, 'loginBtnStd').click()
        self.closepopup(driver)

        driver.find_element(By.CLASS_NAME, 'my_link.status').click()
        driver.find_element(By.ID, 'cns').click()
        time.sleep(1)

        consulting_total = driver.find_element(By.ID, 'pfcCnsCnt').text

        consulting_detail = {'1학기': {'1학년': 0, '2학년': 0, '3학년': 0, '4학년': 0},
                             '2학기': {'1학년': 0, '2학년': 0, '3학년': 0, '4학년': 0}}

        for i in range(1, 3):
            for j in range(1, 5):
                link = f'term{i}_{j}'
                consulting_detail[str(i) + '학기'][str(j) + '학년'] = driver.find_element(By.ID, link).text

        return consulting_total, consulting_detail

    def crawl_object(self):
        filter = ['이수영역', '년도학기', '교과목명', '학점', '성적', '구이수']

        driver = self.driver

        # 로그인 페이지로 이동
        login_url = 'https://www.daejin.ac.kr/subLogin/daejin/view.do?layout=unknown'
        driver.get(login_url)

        # 로그인 폼 채우기
        username_input = driver.find_element(By.ID, 'userId')
        password_input = driver.find_element(By.ID, 'userPwd')

        username_input.send_keys(self.username)
        password_input.send_keys(self.password)

        # 로그인 버튼 클릭
        login_button = driver.find_element(By.CLASS_NAME, '_loginSubmit')
        login_button.click()

        time.sleep(1)

        if 'message.do' in driver.current_url:
            driver.find_element(By.XPATH, '/html/body/div[1]/form/div/div/div[2]/div/span/input').click()

        url = 'https://dreams2.daejin.ac.kr/sugang/LinkPortal.jsp'  # 이동하려는 URL로 수정하세요
        driver.get(url)

        time.sleep(1)
        self.closepopup(driver)

        #테스트 시작
        driver.switch_to.frame('BBF')
        driver.switch_to.frame('LF')
        driver.find_element(By.ID, 'MFX8').click()

        element = driver.find_element(By.ID, 'MFX9')
        element.find_element(By.TAG_NAME, 'a').click()

        #이수구분별 성적조회 및 출력클릭
        driver.switch_to.parent_frame()
        driver.switch_to.frame('RF')
        driver.find_element(By.XPATH, '/html/body/table/tbody/tr/td/table[2]/tbody/tr[3]/td[1]/a').click()
        time.sleep(1)
        driver.find_element(By.CLASS_NAME, "crownix-absolute-100").click() # 화면 세로 맞춤
        driver.find_element(By.ID, "crownix-toolbar-height").click()

        # m2soft-crownix-text ID를 갖는 DIV 요소 찾기
        time.sleep(1)
        div_element = driver.find_element(By.ID, 'm2soft-crownix-text')

        # DIV 요소의 모든 하위 요소들 가져오기
        sub_elements = div_element.find_elements(By.XPATH, ".//*")

        Course = {}
        Course['교필'] = {'학점' : 0, '교과목명' : []}
        Course['교선1'] = {'학점' : 0, '교과목명' : []}
        Course['교선2'] = {'학점' : 0, '교과목명' : []}
        Course['교선3'] = {'학점' : 0, '교과목명' : []}
        Course['교선4'] = {'학점' : 0, '교과목명' : []}
        Course['교선5'] = {'학점' : 0, '교과목명' : []}
        Course['교선6'] = {'학점' : 0, '교과목명' : []}
        Course['교선7'] = {'학점' : 0, '교과목명' : []}
        Course['교선8'] = {'학점' : 0, '교과목명' : []}
        Course['전필'] = {'학점' : 0, '교과목명' : []}
        Course['전선'] = {'학점' : 0, '교과목명' : []}
        Course['일선'] = {'학점' : 0, '교과목명' : []}

        i = 19

        while i < len(sub_elements):
            text = sub_elements[i].text
            if '복전1' in text and '복전1' not in Course:
                Course['복전1'] = {'학점' : 0, '교과목명' : []}
            if '부전1' in text and '부전1' not in Course:
                Course['부전1'] = {'학점' : 0, '교과목명' : []}
            if "일선 취득학점 계 : " in text:
                break
            elif ' 계 : ' in text:
                i += 2
            elif text not in filter:
                if text == '부전1' or text != '':
                    skip = 7 if text == '부전1' else 6
                    
                    if len(text) > 2:
                        text = text[0:3]
                    Course[text]['학점'] += int(sub_elements[i+4].text)
                    Course[text]['교과목명'].append(sub_elements[i+3].text)
                    i += skip
                else:
                    i += 1
            else:
                i += 1
        
        return Course, sub_elements

    def crawling_main(self):
        with ThreadPoolExecutor(max_workers=2) as executor:
            future1 = executor.submit(self.crawl_object)
            future2 = executor.submit(self.crawl_consulting)

            self.Course, self.sub_elements = future1.result()
            self.consulting_total, self.consulting_detail = future2.result()
            
    def require_credit(self):
        # 초기 설정
        Require = {
            '졸업학점': {'기준': 0, '취득': 0},
            '졸업평점평균': {'기준': 0, '취득': 0},
            '교필': {'기준': 0, '취득': 0},
            '교선': {'기준': 0, '취득': 0},
            '전기': {'기준': 0, '취득': 0},
            '전필': {'기준': 0, '취득': 0},
            '전선': {'기준': 0, '취득': 0},
        }

        # '기준'과 '취득' 값 찾기 및 업데이트
        for offset in ['기준', '취득']:
            keyword = '기 준' if offset == '기준' else '취 득'
            for i in range(len(self.sub_elements) - 100, len(self.sub_elements)):
                if keyword in self.sub_elements[i].text:
                    start_index = i + 1
                    break

            keys = ['졸업학점', '졸업평점평균', '교필', '교선', '전기', '전필', '전선']
            for idx, key in enumerate(keys):
                Require[key][offset] = self.sub_elements[start_index + idx].text

            basic = int(Require['전기']['기준'])
            essential = int(Require['전필']['기준'])
            Require['전선']['기준'] = str(63 - basic - essential)

            if '복전1' in self.Course:
                if '복전1' not in Require:
                    Require['복전1'] = {'기준': 0, '취득': 0}
                Require['전선']['기준'] = str(42 - basic - essential) if offset == '기준' else Require['전선']['기준']
                Require['복전1'][offset] = self.sub_elements[start_index + 6 if offset == '취득' else 6].text

        self.Require = Require

    def require_cultural(self):
        for i in range(len(self.sub_elements)-50, len(self.sub_elements)):
            text = self.sub_elements[i].text
            if "기 준" in text:
                start = i+1
            if "취 득" in text:
                end = i
                break

        Require_cultural = {}

        j = 1
        for i in range(start, end):
            Require_cultural["교선" + str(j)] = {'기준': self.sub_elements[i].text, '취득' : self.Course["교선"+str(j)]['학점']}
            j += 1
        
        self.Require_cultural =  Require_cultural

In [2]:
class DaejinCrawler:
    def __init__(self, username, password):
        self.username = username
        self.password = password
        self.Course = {}
        self.Require = {}
        self.Require_cultural = {}
        self.consultng_total = {}
        self.consulting_detail = {}
        self.sub_elements = None
        self.crawling_main()
        self.require_credit()
        self.require_cultural()

    def __del__(self):
        driver.quit()

    def ChromeSetup(self):
        options = webdriver.ChromeOptions()
        #options.add_argument("--headless")
        options.add_argument('--ignore-ssl-errors=yes')
        options.add_argument('--ignore-certificate-errors')
        options.add_argument('--disable-dev-shm-usage')
        options.add_argument('--disable-images')
        options.add_argument("--no-sandbox")
        options.add_argument("--disable-extensions")
        options.add_argument("--disable-popup-blocking")

        return options
    
    def closepopup(self, driver):
        main = driver.window_handles
        for i in main:
            if i != main[0]:
                driver.switch_to.window(i)
                driver.close()

        driver.switch_to.window(main[0])

    def crawl_consulting(self):
        driver = webdriver.Chrome(options=self.ChromeSetup())
        
        twin_url = 'https://together.daejin.ac.kr/clientMain/a/t/main.do'
        driver.get(twin_url)

        self.closepopup(driver)

        driver.find_element(By.CLASS_NAME, 'btn_log.btn_login').click()
        time.sleep(1)
        
        username_input = driver.find_element(By.ID, 'userId')
        password_input = driver.find_element(By.ID, 'userPw')

        username_input.send_keys(self.username)
        password_input.send_keys(self.password)
        driver.find_element(By.ID, 'loginBtnStd').click()
        self.closepopup(driver)

        driver.find_element(By.CLASS_NAME, 'my_link.status').click()
        driver.find_element(By.ID, 'cns').click()
        time.sleep(1)

        consulting_total = driver.find_element(By.ID, 'pfcCnsCnt').text

        consulting_detail = {'1학기': {'1학년': 0, '2학년': 0, '3학년': 0, '4학년': 0},
                             '2학기': {'1학년': 0, '2학년': 0, '3학년': 0, '4학년': 0}}

        for i in range(1, 3):
            for j in range(1, 5):
                link = f'term{i}_{j}'
                consulting_detail[str(i) + '학기'][str(j) + '학년'] = driver.find_element(By.ID, link).text

        return consulting_total, consulting_detail

    def crawl_object(self):
        filter = ['이수영역', '년도학기', '교과목명', '학점', '성적', '구이수']

        driver = webdriver.Chrome(options=self.ChromeSetup())

        # 로그인 페이지로 이동
        login_url = 'https://www.daejin.ac.kr/subLogin/daejin/view.do?layout=unknown'
        driver.get(login_url)

        # 로그인 폼 채우기
        username_input = driver.find_element(By.ID, 'userId')
        password_input = driver.find_element(By.ID, 'userPwd')

        username_input.send_keys(self.username)
        password_input.send_keys(self.password)

        # 로그인 버튼 클릭
        login_button = driver.find_element(By.CLASS_NAME, '_loginSubmit')
        login_button.click()

        time.sleep(1)

        if 'message.do' in driver.current_url:
            driver.find_element(By.XPATH, '/html/body/div[1]/form/div/div/div[2]/div/span/input').click()

        url = 'https://dreams2.daejin.ac.kr/sugang/LinkPortal.jsp'  # 이동하려는 URL로 수정하세요
        driver.get(url)

        time.sleep(1)
        self.closepopup(driver)

        #테스트 시작
        driver.switch_to.frame('BBF')
        driver.switch_to.frame('LF')
        driver.find_element(By.ID, 'MFX8').click()

        element = driver.find_element(By.ID, 'MFX9')
        element.find_element(By.TAG_NAME, 'a').click()

        #이수구분별 성적조회 및 출력클릭
        driver.switch_to.parent_frame()
        driver.switch_to.frame('RF')
        driver.find_element(By.XPATH, '/html/body/table/tbody/tr/td/table[2]/tbody/tr[3]/td[1]/a').click()
        time.sleep(1)
        driver.find_element(By.CLASS_NAME, "crownix-absolute-100").click() # 화면 세로 맞춤
        driver.find_element(By.ID, "crownix-toolbar-height").click()

        # m2soft-crownix-text ID를 갖는 DIV 요소 찾기
        time.sleep(1)
        div_element = driver.find_element(By.ID, 'm2soft-crownix-text')

        # DIV 요소의 모든 하위 요소들 가져오기
        sub_elements = div_element.find_elements(By.XPATH, ".//*")

        Course = {}
        Course['교필'] = {'학점' : 0, '교과목명' : []}
        Course['교선1'] = {'학점' : 0, '교과목명' : []}
        Course['교선2'] = {'학점' : 0, '교과목명' : []}
        Course['교선3'] = {'학점' : 0, '교과목명' : []}
        Course['교선4'] = {'학점' : 0, '교과목명' : []}
        Course['교선5'] = {'학점' : 0, '교과목명' : []}
        Course['교선6'] = {'학점' : 0, '교과목명' : []}
        Course['교선7'] = {'학점' : 0, '교과목명' : []}
        Course['교선8'] = {'학점' : 0, '교과목명' : []}
        Course['전필'] = {'학점' : 0, '교과목명' : []}
        Course['전선'] = {'학점' : 0, '교과목명' : []}
        Course['일선'] = {'학점' : 0, '교과목명' : []}

        i = 19

        while i < len(sub_elements):
            text = sub_elements[i].text
            if '복전1' in text and '복전1' not in Course:
                Course['복전1'] = {'학점' : 0, '교과목명' : []}
            if '부전1' in text and '부전1' not in Course:
                Course['부전1'] = {'학점' : 0, '교과목명' : []}
            if "일선 취득학점 계 : " in text:
                break
            elif ' 계 : ' in text:
                i += 2
            elif text not in filter:
                if text == '부전1' or text != '':
                    skip = 7 if text == '부전1' else 6
                    
                    if len(text) > 2:
                        text = text[0:3]
                    Course[text]['학점'] += int(sub_elements[i+4].text)
                    Course[text]['교과목명'].append(sub_elements[i+3].text)
                    i += skip
                else:
                    i += 1
            else:
                i += 1
        
        return Course, sub_elements

    def crawling_main(self):
        with ThreadPoolExecutor(max_workers=2) as executor:
            future1 = executor.submit(self.crawl_object)
            future2 = executor.submit(self.crawl_consulting)

            self.Course, self.sub_elements = future1.result()
            self.consulting_total, self.consulting_detail = future2.result()

    def require_credit(self):
        # 초기 설정
        Require = {
            '졸업학점': {'기준': 0, '취득': 0},
            '졸업평점평균': {'기준': 0, '취득': 0},
            '교필': {'기준': 0, '취득': 0},
            '교선': {'기준': 0, '취득': 0},
            '전기': {'기준': 0, '취득': 0},
            '전필': {'기준': 0, '취득': 0},
            '전선': {'기준': 0, '취득': 0},
        }

        # '기준'과 '취득' 값 찾기 및 업데이트
        for offset in ['기준', '취득']:
            keyword = '기 준' if offset == '기준' else '취 득'
            for i in range(len(self.sub_elements) - 100, len(self.sub_elements)):
                if keyword in self.sub_elements[i].text:
                    start_index = i + 1
                    break

            keys = ['졸업학점', '졸업평점평균', '교필', '교선', '전기', '전필', '전선']
            for idx, key in enumerate(keys):
                Require[key][offset] = self.sub_elements[start_index + idx].text

            basic = int(Require['전기']['기준'])
            essential = int(Require['전필']['기준'])
            Require['전선']['기준'] = str(63 - basic - essential)

            if '복전1' in self.Course:
                if '복전1' not in Require:
                    Require['복전1'] = {'기준': 0, '취득': 0}
                Require['전선']['기준'] = str(42 - basic - essential) if offset == '기준' else Require['전선']['기준']
                Require['복전1'][offset] = self.sub_elements[start_index + 6 if offset == '취득' else 6].text
        self.Require = Require

    def require_cultural(self):
        for i in range(len(self.sub_elements)-50, len(self.sub_elements)):
            text = self.sub_elements[i].text
            if "기 준" in text:
                start = i+1
            if "취 득" in text:
                end = i
                break

        Require_cultural = {}

        j = 1
        for i in range(start, end):
            Require_cultural["교선" + str(j)] = {'기준': self.sub_elements[i].text, '취득' : self.Course["교선"+str(j)]['학점']}
            j += 1
        
        self.Require_cultural =  Require_cultural

In [3]:
daejin = DaejinCrawler('20191516', 'cho6090^)()')
daejin.Course

{'교필': {'학점': 8,
  '교과목명': ['비전설계및상담A', '사고와표현', '비전설계및상담A', '영어읽기와토론', '대순사상의이해']},
 '교선1': {'학점': 0, '교과목명': []},
 '교선2': {'학점': 2, '교과목명': ['음식과식량의문화사']},
 '교선3': {'학점': 2, '교과목명': ['스포츠마케팅인사이트']},
 '교선4': {'학점': 4, '교과목명': ['알기쉬운빅데이터와통계학의이해', '정보사회와컴퓨터']},
 '교선5': {'학점': 2, '교과목명': ['서양음악의역사와감상']},
 '교선6': {'학점': 0, '교과목명': []},
 '교선7': {'학점': 5, '교과목명': ['토익문법', '명사와함께하는나의미래설계']},
 '교선8': {'학점': 14,
  '교과목명': ['빅데이터분석의이해', '기초통계학', '제4차산업혁명과자기경영', 'OFFICE실무', '성격심리학']},
 '전필': {'학점': 12, '교과목명': ['컴퓨터시스템구조', '운영체제론', '컴퓨터알고리즘', '데이터베이스론']},
 '전선': {'학점': 48,
  '교과목명': ['컴퓨터프로그래밍기초',
   '콘텐츠기술의이해',
   '창의적공학프로그래밍입문',
   '객체지향프로그래밍',
   '컴퓨터시스템기초실습',
   '전산수학',
   '논리회로및실습',
   '자료구조',
   '시스템프로그래밍',
   '파일처리론',
   '자바프로그래밍',
   '데이터베이스설계',
   '고급시스템프로그래밍',
   '컴퓨터네트워크및보안',
   '웹프로그래밍',
   '프론트엔드웹디자인']},
 '일선': {'학점': 12, '교과목명': ['기초프로그래밍', '빅데이터분석심화', '정형DB실무', 'AI개론']},
 '부전1': {'학점': 18,
  '교과목명': ['원가.관리회계', '회계원리', '기업재무', '서비스운영관리', '인적자원관리', '조직행동론']}}

In [4]:
del daejin

Exception ignored in: <function DaejinCrawler.__del__ at 0x00000258E8B7B6A0>
Traceback (most recent call last):
  File "C:\Users\cho03\AppData\Local\Temp\ipykernel_14660\298223849.py", line 16, in __del__
NameError: name 'driver' is not defined


In [ ]:
daejin.Course

In [6]:
daejin.Require_cultural

{'교선1': {'기준': '1', '취득': 0},
 '교선2': {'기준': '1', '취득': 2},
 '교선3': {'기준': '1', '취득': 2},
 '교선4': {'기준': '1', '취득': 4},
 '교선5': {'기준': '1', '취득': 2}}

In [7]:
daejin.Require

AttributeError: 'DaejinCrawler' object has no attribute 'Require_credit'